In [8]:
import cv2 as cv
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
#from pixelmatch import pixelmatch as pm
from pixelmatch.contrib.PIL import pixelmatch as pm2
import os
from pathlib import *
#import magickwand as mw

In [9]:
img = Image.open("images/test1.gif")
gt = Image.open("images/ground_truth.gif")

In [10]:
img_diff = Image.new("RGB", img.size)
width, height= img.width, img.height

Creating an image mask, the mismatching pixel will be returned on red color (i.e. rgb[255, 0, 0])

In [54]:
#mismatch = pm2(img, gt, img_diff, threshold=.2, includeAA=True, diff_color=[0, 0, 0])

#Mask generation
#mismatch = pm2(img, gt, img_diff, threshold=0, alpha=0.4, includeAA=True)
mismatch = pm2(img, gt, img_diff, threshold=0, includeAA=True, diff_mask=True)

#img_diff.save("diff.png")

#Mask saving
img_diff.save("diff_red.png")

Splitting the original image in multiple images, one for each line using the script *multicrop2*

In [49]:
!sh multicrop2 diff_red.png lines/line.png

multicrop2: line 215: type: multicrop2: not found
usage: dirname string [...]
usage: basename string [suffix]
       basename [-a] [-s suffix] string [...]


Processing Image 0
Initial Crop Box: 425x17+283+969

Processing Image 1
Initial Crop Box: 423x17+289+831

Processing Image 2
Initial Crop Box: 420x14+281+1097

Processing Image 3
Initial Crop Box: 418x11+279+1019

Processing Image 4
Initial Crop Box: 418x14+281+1077

Processing Image 5
Initial Crop Box: 417x13+284+1039

Processing Image 6
Initial Crop Box: 411x11+284+1118

Processing Image 7
Initial Crop Box: 409x15+283+1057

Processing Image 8
Initial Crop Box: 402x14+288+453

Processing Image 9
Initial Crop Box: 398x18+289+804

Processing Image 10
Initial Crop Box: 397x17+282+695

Processing Image 11
Initial Crop Box: 393x21+289+337

Processing Image 12
Initial Crop Box: 390x13+292+202

Processing Image 13
Initial Crop Box: 385x14+286+479

Processing Image 14
Initial Crop Box: 382x18+282+942

Processing Image 15
Initial Crop Box

In [55]:
#Aplha channel integration on the lines images
def alpha_channel2(image, dir='/Users/mappo/PycharmProjects/DIA/DIA-Project/'):
    img = Image.open(image)
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 153 and item[1] == 153 and item[2] == 153:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)
    name, extension = os.path.splitext(image)
    string = dir + os.path.basename(name) + '_alpha.png'
    img.save(string, "PNG")

In [56]:
alpha_channel2('diff_red.png')

In [ ]:
#Aplha channel integration on the lines images
"""
def alpha_channel(image, dir='/Users/mappo/PycharmProjects/DIA/DIA-Project/lines_alpha/'):
    img = Image.open(image)
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 229 and item[1] == 229 and item[2] == 229:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)
    name, extension = os.path.splitext(image)
    string = dir + os.path.basename(name) + '_alpha.png'
    img.save(string, "PNG")
"""

In [5]:
#os.path.isfile('./lines/line-000.png')
#os.path.isdir('/Users/mappo/PycharmProjects/DIA/DIA-Project/lines')
#pathlib.Path()
#p = pathlib.Path("lines", "line-000.png")

NameError: name 'pathlib' is not defined

In [32]:
#alpha_channel('./lines/line-000.png')

In [18]:
#src_dir = '/Users/mappo/PycharmProjects/DIA/DIA-Project/lines'
#dst_dir = '/Users/mappo/PycharmProjects/DIA/DIA-Project/lines_alpha'

In [20]:
#for filename in os.listdir(src_dir):
#    alpha_channel(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'line-057.png'

In [3]:
#img = cv.imread("line-000.png")
#img2 = cv.imread("line-000_alpha.png")

Definig a function able to find number of pixels of a certain colour
Adapted from: <a href=https://github.com/townsean/pixel-color-count/blob/master/pixel-color-count.py>https://github.com/townsean/pixel-color-count/blob/master/pixel-color-count.py<a>

In [61]:
def count_pixels(filename):
    """
    Returns a count of pixels per a unique color
    Args:
        filename (str): the image to count the number of pixels of
    Returns:
        a key-value pairing of the rgb color value and the number of times the color was present in the image
    """
    color_count = {}
    with Image.open(filename) as image:
        width, height = image.size
        rgb_image = image.convert('RGBA')

        # iterate through each pixel in the image and keep a count per unique color
        for x in range(width):
            for y in range(height):
                rgb = rgb_image.getpixel((x, y))

                if rgb in color_count:
                    color_count[rgb] += 1
                else:
                    color_count[rgb] = 1

    return color_count

In [62]:
colors = count_pixels('./lines2/line-000.png')
#colors = count_pixels('./lines_alpha/line-000_alpha.png')

In [63]:
colors

{(255, 0, 0, 255): 862, (255, 255, 255, 0): 2972, (255, 255, 255, 255): 3391}

In [85]:
type(colors)

dict

In [64]:
red_pxl = colors[(255, 0, 0, 255)]
w_pxl = colors[(255, 255, 255, 255)]

Accuracy calculation

In [68]:
acc = w_pxl / (w_pxl + red_pxl)
print("The accuracy for line 000 is: " + str(acc))

The accuracy for line 000 is: 0.7973195391488361


In [101]:
def element_acc(img):
    colors = count_pixels(img)
    #mismatched pixel are on red colour
    #red_pxl = if(colors[(255, 0, 0, 255)]) else return 0
    red_pxl = '(255, 0, 0, 255)' in colors.values()
    #correctly mached ones are white
    #w_pxl = colors[(255, 255, 255, 255)]
    w_pxl = '(255, 255, 255, 255)' in colors.values()
    #Accuracy param calculation right_pred_pixels / (right_pred_pixels + wrongly_pred_pixels)
    #standard case
    if (w_pxl & red_pxl) != 0:
        acc = w_pxl / (w_pxl + red_pxl)
    #case where red pixels are 0
    elif(red_pxl == 0 & w_pxl != 0):
        acc = 1
    #case where white pixels are 0
    else:
        acc = 0
    return(acc)

In [ ]:
element_acc()

In [103]:
folder_dir = "/Users/mappo/PycharmProjects/DIA/DIA-Project/lines2/"
counter = 0

for images in os.listdir(folder_dir):
    # check if the image ends with png
    if (images.endswith(".png")):
        counter += 1
        acc += element_acc(folder_dir + images)

print("We have calulated the accuracy of "+ str(counter) + " different lines featured on the image. \n")
avg_acc = acc/counter
print("The accuracy for the image is: " + str(avg_acc))

We have calulated the accuracy of 92 different lines featured on the image. 

The accuracy for the image is: 0.16375177149313402
